In [2]:
import pandas as pd
import numpy as np
PATH = '/home/kai/data/kaggle/homecredit/'

# Read train and test
train = pd.read_pickle(PATH + 'train_factorized_1_with_linear.pkl')
test = pd.read_pickle(PATH + 'test_factorized_1_with_linear.pkl')
train.shape, test.shape

((307511, 1918), (48744, 1917))

In [3]:
def model_2(train_df, test_df, threshold1, threshold2, n_splits):
    print('begin cv')
    target = train_df['TARGET']
    ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
    features = [x for x in train_df.columns if x not in ignore_cols]
    train = train_df[features]
    test = test_df[features]

    from sklearn.model_selection import StratifiedKFold
    from lightgbm import LGBMClassifier
    from sklearn.metrics import roc_auc_score
    import lightgbm as lgbm

    lgbm_train = lgbm.Dataset(data=train,
                              label=target,

                              categorical_feature=[],#categorical_feats,
                              free_raw_data=False)
    
    lgbm_params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_iteration': 4000,
        'num_threads': 8,
        
        'num_leaves': int(round(44.368535336628419)),
        'feature_fraction': 0.28231763168020257,
        'bagging_fraction': 0.94901525271474951,
        'max_depth': int(round(8.0430115561596267)),
        'lambda_l1': 0.30680079516647751,
        'lambda_l2': 0.079128660903201031,
        'min_split_gain': 0.054005067457890979,
        'min_child_weight': 98.172643147364937
    }

    cv_results = lgbm.cv(train_set=lgbm_train,
                         params=lgbm_params,
                         nfold=n_splits,
                         seed=2018,
                         early_stopping_rounds=150,
                         verbose_eval=100,
                         metrics=['auc'])

    optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
    print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
    print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

    clf = lgbm.train(train_set=lgbm_train,
                     params=lgbm_params,
                     num_boost_round=optimum_boost_rounds)

    """ Predict on test set and create submission """
    y_pred = clf.predict(test)
    out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})
    small_len = out_df[out_df['TARGET']< threshold1]
    large_len = out_df[out_df['TARGET']> threshold2]
    print('length of small and large,',len(small_len),len(large_len))
    return out_df, small_len, large_len 

In [ ]:
threshold1, threshold2 = 0.005, 0.5
n_splits = 5

pred1, small, large = model_2(train, test, threshold1, threshold2,n_splits) 

begin cv


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.777611 + 0.00437966
[200]	cv_agg's auc: 0.784903 + 0.00415943
[300]	cv_agg's auc: 0.786765 + 0.00370779
[400]	cv_agg's auc: 0.787215 + 0.00346352
[500]	cv_agg's auc: 0.787577 + 0.00352841


1